In [ ]:
pip install selenium

In [ ]:
pip install webdriver_mamager

In [ ]:
pip install undetected_chromedriver

In [ ]:
pip install chromedriver_autoinstaller

In [ ]:
pip install pyautogui

In [ ]:
pip install openpyxl

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import subprocess
import shutil
import pyautogui

In [ ]:
#google.txt에서 아디 비번 실시간으로 받아옴
print("google 텍스트 문서에서 아이디와 비밀번호, 저장할 파일 이름 정보를 가져옵니다.\n")
f=open('google.txt','r', encoding='UTF8') #한글 가능
line = f.readlines()
google_idraw = line[1]
google_id = google_idraw.replace('\n','')
google_pwraw = line[3]
google_pw = google_pwraw.replace('\n','')
color_code_raw = line[5]
color_code = color_code_raw.replace('\n','')
filename = line[7]
print("크롤링 할 구글 아이디 : " + google_id + "\n크롤링 할 구글 아이디의 비밀번호 : " + google_pw + "\n" )
print("파일 이름은 : " + filename + ".xlsx로 저장될 예정입니다.\n" )
print("시작 전 모든 크롬 창을 닫고 진행하시는 것을 권장드립니다. 혹은 새 창이 생기면 빠르게 새 창을 클릭해주세요.\n")

In [ ]:
#구글 로그인 우회
try:
    shutil.rmtree(r"C:\chrometemp")  # remove Cookie, Cache files
except FileNotFoundError:
    pass

try:
    subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 '
                     r'--user-data-dir="C:\chrometemp"')   # Open the debugger chrome
    
except FileNotFoundError:
    print('error')

option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
    
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
driver.implicitly_wait(10)

driver.get(
    url='https://accounts.google.com/signin/chrome/sync/identifier?ssp=1&continue=https%3A%2F%2Fwww.google.com%2F&flowName=GlifDesktopChromeSync')
# 구글 로그인 페이지 긁어온거
driver.switch_to.window(driver.window_handles[-1]) # 최근 열린 탭으로 전환

pyautogui.write(google_id)    # 메모장의 아이디 부분
pyautogui.press('tab', presses=3)
pyautogui.press('enter')
time.sleep(3)
pyautogui.write(google_pw)   # 메모장의 비밀번호 부분
pyautogui.press('enter')

time.sleep(3)
import requests
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import openpyxl
from openpyxl.styles import Color, PatternFill

#EXcel Sheet 생성
wb = openpyxl.Workbook()

sheet = wb.active
sheet.append(["날짜", "결제 계정", "상품명", "화폐 단위", "금액"])
#맨 윗줄 셀 색칠하기
a1 = sheet['A1']
b1 = sheet['B1']
c1 = sheet['C1']
d1 = sheet['D1']
e1 = sheet['E1']
a1.fill = PatternFill(fill_type='solid',
                      fgColor=Color(color_code))
b1.fill = PatternFill(fill_type='solid',
                      fgColor=Color(color_code))
c1.fill = PatternFill(fill_type='solid',
                      fgColor=Color(color_code))
d1.fill = PatternFill(fill_type='solid',
                      fgColor=Color(color_code))
e1.fill = PatternFill(fill_type='solid',
                      fgColor=Color(color_code))

raw = driver.get("https://play.google.com/store/account/orderhistory")

time.sleep(3)

# body태그의 내부 HTML 코드를 가져오고 그 코드를 BeautifulSoup로 파싱한다.
htmlText = driver.find_element(By.TAG_NAME, 'body').get_attribute('innerHTML')
html = BeautifulSoup(htmlText, 'html.parser')

# data-item-type 속성을 가진 div 태그들을 가져온다.
container = html.select("div[data-item-type]")

# 정규표현식 이용
# * ([^0-9\\.]+) 뜻 : 숫자와 온점이 "아닌" 모든 문자들의 집합
# * ([0-9\\.,]+) 뜻 : 숫자와 온점 콤마"으로만" 이루어진 모든 문자들의 집합
import re
pricePattern = re.compile("([^0-9\\.]+)([0-9\\.,]+)")

for con in container:
    a = con.select_one(".V8HwNc").text.strip() #날짜
    b = google_id # 결제 계정
    c = con.select_one(".XqqpEd").text.strip() #상품명
    raw_cash = con.select_one(".mshXob").text.strip() #화폐 단위 + 금액
    strings = pricePattern.match(raw_cash).groups() # CA$17.99 라고 받는다 치면 앞뒤로 나눔
    d = strings[0]
    e = strings[1]
    
    # sheet 내 각 행에 데이터 추가
    sheet.append([a, b, c, d, e])

wb.save(filename +".xlsx")
print("크롤링 성공")